In [1]:
import os 
import glob
from ase.io import read,write
from ovito.io import import_file
import WarrenCowleyParameters as wc

simulation_folder = '/nfshome/winkelmann/CathodeSimulationResults/Lichtenberg_X-mas_25_26'

# find Ovito Files

In [4]:
pattern = os.path.join(simulation_folder, '**', '*.XDATCAR')
path_all_ovito_files = sorted(glob.glob(pattern, recursive=True))

# Find all truncated files and create list of their original files
truncated_files = [f for f in path_all_ovito_files if f.endswith('_truncated.XDATCAR')]
original_files_to_exclude = []#[f.replace('_truncated.XDATCAR', '.XDATCAR') for f in truncated_files]

# Exclude both truncated files and their base files
path_all_ovito_files = [f for f in path_all_ovito_files if f not in truncated_files and f not in original_files_to_exclude]
print(f"Found {len(path_all_ovito_files)} ovito files to process.")
print(f'excluded {len(truncated_files)} truncated files and their base files.')
print('\n'.join(f'{i+1}. {f}' for i, f in enumerate(sorted(original_files_to_exclude))))

Found 180 ovito files to process.
excluded 180 truncated files and their base files.



# truncate to only Li and X Atoms

In [7]:
#todo: find a way to keep the labels of the frames
def truncate_ovito_files(path_ovito_file, wanted_species=['Li','X']):
    all_frames = read(path_ovito_file, index=':')
    truncated_ovito_file = []
    for atoms in all_frames:  
        indices = [i for i, atom in enumerate(atoms) if atom.symbol in wanted_species]
        new_atoms = atoms[indices]
        new_atoms.info = atoms.info
        truncated_ovito_file.append(new_atoms)      
    write(path_ovito_file.replace('.XDATCAR', '_truncated.XDATCAR'), truncated_ovito_file)
    return truncated_ovito_file

for i, path_ovito_file in enumerate(path_all_ovito_files):
    progress = (i + 1) / len(path_all_ovito_files) * 100
    print(f'Progress: {i/len(path_all_ovito_files) * 100:.0f}% \t Processing File {i+1}/{len(path_all_ovito_files)}', end='\r', flush=True)
    truncate_ovito_files(path_ovito_file)

KeyboardInterrupt: 

# find trunicated files ()

In [2]:
pattern = os.path.join(simulation_folder, '**', '*_truncated.XDATCAR')
path_truncated_files = sorted(glob.glob(pattern, recursive=True))

# calculate Warren Cowley and safe it to a CSV

In [ ]:
def create_warren_cowley_csv(structure_file):
    ovito_file = import_file(structure_file)
    
    num_frames = ovito_file.source.num_frames
    mod = wc.WarrenCowleyParameters(nneigh=[0, 6])
    ovito_file.modifiers.append(mod)
    
    data = ovito_file.compute(0)
    wc_names = list(data.attributes['Warren-Cowley parameters by particle name'][0].keys())
    
    wc_file = structure_file.replace('.XDATCAR', '_warren_cowley.csv')
    with open(wc_file, 'w') as f:
        f.write(f"step,\ttime,\t{wc_names[0]},\t{wc_names[1]},\t{wc_names[2]},\t{wc_names[3]}\n")
    # Iterate through frames
    for frame in range(num_frames):
        data = ovito_file.compute(frame)
        wc_for_shells = data.attributes['Warren-Cowley parameters'][0]
        with open(wc_file, 'a') as f:
            f.write(f"Frame {frame},\t{wc_for_shells[0][0]},\t{wc_for_shells[0][1]},\t{wc_for_shells[1][0]},\t{wc_for_shells[1][1]}\n")
    

for i, path in enumerate(path_truncated_files):
    progress = (i + 1) / len(path_truncated_files) * 100
    print(f'Progress: {i/len(path_truncated_files) * 100:.0f}% \t Processing File {i+1}/{len(path_truncated_files)}', end='\r', flush=True)
    create_warren_cowley_csv(path)
    

SourceFrame = 78rocessing File 2/180
SourceFile = /nfshome/winkelmann/CathodeSimulationResults/Lichtenberg_X-mas_25_26/Li_12.5/02/kmc_Cathode_Ovito_truncated.XDATCAR
Comment = Li X
Warren-Cowley parameters = [[[ 0.59876543 -0.08553792]
  [-0.08553792  0.0122197 ]]]
Warren-Cowley parameters by particle name = [{'Li-Li': 0.5987654320987654, 'Li-X': -0.08553791887125216, 'X-Li': -0.08553791887125212, 'X-X': 0.012219702695893182}]


In [ ]:
    print(ovito_file)
    mod = wc.WarrenCowleyParameters(nneigh=[0, 6])
    ovito_file.modifiers.append(mod)
    
    # Iterate through all frames
    num_frames = ovito_file.source.num_frames
    for frame_idx in range(num_frames):
        data = ovito_file.compute(frame_idx)
        wc_for_shells = data.attributes['Warren-Cowley parameters']
        print(f"Frame {frame_idx}: 1NN Warren-Cowley parameters: \n {wc_for_shells[0]}")